Armijo Line-Search Rule

The successive reduction rule suitably modified to eliminate theoretical convergence difficulty

Featuring 'Newton's Method' and 'Steepest Decent' applied to a logistic regression problem.

In [42]:
#Imports and definitions
import numpy as np
from pandas import DataFrame as df
from pandas import concat as cc

#Initial values
beta0,beta1 = 5,-2
init_p = [beta0,beta1]

#Data
hours = [0.50,0.75,1.00,1.25,1.50,1.75,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,4.00,4.25,4.50,4.75,5.00,5.50]
passed = [0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1]

#Line search parameters
s,beta,sigma = 1,0.5,0.1

#Number of iterations
n_iter= range(5)

#logreg prob
def p(x,beta0=beta0,beta1=beta1):
    ex = beta0 + beta1*x
    px = 1/(1+np.exp(-ex))
    return px

#Function to be minimized/Loglikelihood
def loglik(x_list=hours,y_list=passed,beta0=beta0,beta1=beta1):
    cumsum = 0
    for i,x in enumerate(x_list):
        cumsum += y_list[i]*np.log(p(x,beta0=beta0,beta1=beta1)) 
        cumsum += (1-y_list[i])*np.log(1-p(x,beta0=beta0,beta1=beta1))
    return cumsum

#Gradient of loglikelihood function
def loglik_grad(x_list=hours,y_list=passed,beta0=beta0,beta1=beta1):
    dl_db0 = 0
    dl_db1 = 0

    for i,x in enumerate(x_list):
        dl_db0 += (y_list[i]-p(x,beta0=beta0,beta1=beta1))
        dl_db1 += (y_list[i]-(x*p(x,beta0=beta0,beta1=beta1)))
    
    return np.array([dl_db0,dl_db1])

def loglik_hess(x_list=hours,y_list=passed,beta0=beta0,beta1=beta1):
    d2_db02 = 0
    d2_db0b1 = 0
    d2_db12 = 0

    for i,x in enumerate(x_list):
        d2_db02 += (y_list[i]-(p(x,beta0=beta0,beta1=beta1)*(1-p(x,beta0=beta0,beta1=beta1))))
        d2_db0b1 += (y_list[i]-(x*p(x,beta0=beta0,beta1=beta1)*(1-p(x,beta0=beta0,beta1=beta1))))
        d2_db12 += (y_list[i]-((x**2)*p(x,beta0=beta0,beta1=beta1)*(1-p(x,beta0=beta0,beta1=beta1))))
    return np.array([[d2_db02,d2_db0b1],[d2_db0b1,d2_db12]])


In [44]:
#Newton's Method
def newton(fm=loglik,fm_grad=loglik_grad,fm_hess=loglik_hess,
                init_p=[beta0,beta1],n_iter=n_iter,
                s=s,beta=beta,sigma=sigma):
    """
    Inputs:
        fm: function to minimize
        fm_grad: gradient of fm
        fm_hess: hessian of fm
        init_p: initial point
        n_iter: number of iterations
        s,beta,sigma: line search parameters
    """

    #Lists to store function in/out values
    points_list = []
    points_list.append(init_p)
    fm_list = []
    fm_list.append(fm(beta0=init_p[0],beta1=init_p[1]))
    iter_list = []

    for n in n_iter:
        cur_point = points_list[-1]
        beta0,beta1 = cur_point[0],cur_point[1]

        grad = loglik_grad(beta0=beta0,beta1=beta1)
        hess = loglik_hess(beta0=beta0,beta1=beta1)
        inv_hess = np.linalg.inv(hess)
        delt = np.matmul(inv_hess,grad)

        new_p = [cur_point[i]-delt[i] for i in range(len(cur_point))]
        points_list.append(new_p)
        fm_list.append(fm(beta0=new_p[0],beta1=new_p[1]))

        #Saving iteration values in dicts for later display
        iter = {
                "Method":"Newton's",
                "Step#":n+1,
                "WhileIter":"na",
                "CurPoint":[round(a,3) for a in new_p],
                "F()":fm_list[-1],
            }
        iter_list.append(iter)
        #print(iter)
    return iter_list

In [46]:
#Steepest Ascent
def armijostep(fm=loglik,fm_grad=loglik_grad,
                init_p=[beta0,beta1],n_iter=n_iter,
                s=s,beta=beta,sigma=sigma):
    """
    Inputs:
        fm: function to minimize
        fm_grad: gradient of fm
        init_p: initial point
        n_iter: number of iterations
        s,beta,sigma: line search parameters
    """
    #Number of dimensions determined by length of initiating vector
    n_dim = len(init_p)

    #Lists to store function in/out values
    points_list = []
    points_list.append(init_p)
    fm_list = []
    fm_list.append(fm(beta0=init_p[0],beta1=init_p[1]))
    iter_list = []

    for step in n_iter:
        #Each loop is initialized using the preceding output
        cur_p = points_list[-1]
        last_fm = fm_list[-1]
        
        #The current point is run through the gradient function
        #   We want the direction of steepest descent,
        #   therefore we want the opposite of each value in the gradient
        grad = fm_grad(beta0=cur_p[0],beta1=cur_p[1])
        #d = [-m for m in grad]

        #While loop is initialized with beta**k = 1
        k = 0
        step_size = [grad[i]*s*sigma*(beta**k) for i in range(n_dim)]
        trial_p = [cur_p[j] + step_size[j] for j in range(n_dim)]
        
        #Each loop, if the function returns a larger value, 
        #the step size at each point is reduced by a factor of beta, then rerun
        while fm(beta0=trial_p[0],beta1=trial_p[1]) < last_fm:
            k += 1
            step_size = [grad[i]*s*sigma*(beta**k) for i in range(n_dim)]
            trial_p = [cur_p[j] + step_size[j] for j in range(n_dim)]

        #Upon reaching smaller fm value, the parameters are appended to lists
        points_list.append(trial_p)
        fm_list.append(fm(beta0=trial_p[0],beta1=trial_p[1]))

        #Saving iteration values in dicts for later display
        iter = {
                "Method":"Steepest",
                "Step#":step+1,
                "WhileIter":k,
                "CurPoint":[round(a,3) for a in trial_p],
                "F()":fm_list[-1],
            }
        iter_list.append(iter)
        #print(iter)
    return iter_list

In [47]:
#Dataframe for results display
steepest = armijostep()
newtons = newton()

steep_df = df.from_records(steepest)
newt_df = df.from_records(newtons)

result = cc([steep_df,newt_df])
print(result)


     Method  Step# WhileIter         CurPoint         F()
0  Steepest      1        49      [5.0, -2.0]  -48.918024
1  Steepest      2        51      [5.0, -2.0]  -48.918024
2  Steepest      3        51      [5.0, -2.0]  -48.918024
3  Steepest      4        51      [5.0, -2.0]  -48.918024
4  Steepest      5        51      [5.0, -2.0]  -48.918024
0  Newton's      1        na  [5.434, -2.831]  -69.969386
1  Newton's      2        na  [5.682, -3.595]  -92.443039
2  Newton's      3        na  [5.343, -3.833] -102.076926
3  Newton's      4        na  [4.258, -3.366]  -92.699014
4  Newton's      5        na  [2.917, -2.702]  -78.546203
